In [6]:
import math

## Calculate Magnetic field at position P

In [36]:

def get_B(P):
    R = 0.7
    A = [0.39, 0.4, 0.5]
    P = [0.2, -0.5]
    B = [1,0.5,2]
    G = [0, -2, 0.3]
    
    a = A
    A = []
    curr = 0
    for i in range(len(a)):
        A.append([curr, curr + float(a[i])])
        curr += float(a[i])
    
    # Step 1: check what point it is
    x = P[0]
    y = P[1]

    # Initialize output
    Bout = 0
    
    # Update output, if needed
    for k in range(len(A)):
        m1 = math.tan(math.pi/2 - float(A[k][0]))
        m2 = math.tan(math.pi/2 - float(A[k][1]))
        if y < m1*x - R and y >= m2*x - R:  # if P is in Area k
            d = math.sqrt(x**2 + (y-(-R))**2)
            h = R - d

            Bout = float(B[k])+ float(G[k])*h
            break


    return Bout

In [70]:

B = 5*[get_B([0.4, -0.3])]
B


[-0.3343145750507619,
 -0.3343145750507619,
 -0.3343145750507619,
 -0.3343145750507619,
 -0.3343145750507619]

## Beam Rigidity

In [42]:
B_e = 5*[3.3356 * 100]

In [48]:
B_e

[333.56, 333.56, 333.56, 333.56, 333.56]

## Radius

In [51]:
R = []
for i in range(len(B_e)):
    R.append(B_e[i] / B[i])
R = R[0] 

## New position

In [ ]:
R = [[]]

In [81]:
math.sin(90)

0.8939966636005579

In [82]:
-1+math.cos(-0.01)

-4.999958333473664e-05

In [83]:
import numpy as np

In [87]:
CP2 = np.array([[math.cos(-0.01), -math.sin(-0.01)],
              [math.sin(-0.01), math.cos(-0.01)]])

D = np.array([0, 1])
np.matmul(CP2, D)


array([0.00999983, 0.99995   ])

In [88]:
a = np.array([[0, 1],
              [-1, 0]])

D = np.array([1, 0])
np.matmul(a, D)

array([ 0, -1])

In [94]:
import numpy as np
import math

CP2 = np.array([[-math.sin(-0.01)], [-1 + math.cos(-0.01)]])
D = np.array([0, -1])

result = np.add(D, CP2.flatten())
print(result)

[ 0.00999983 -1.00005   ]
